In [1]:
# dependencies
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import cv2
import os 
import requests
from math import floor

# Read in video game data set
df_0 = pd.read_csv("boris_data.csv")

In [3]:
# clean and get urls 
almost_clean_df= df_0.drop_duplicates(subset = ['background_image'])
clean_df = almost_clean_df.dropna(subset = ['background_image'])

print(clean_df.shape)
URLS = list(clean_df['background_image'])
game_names = list(clean_df['name'])

(23989, 63)


In [4]:
# Function does the following work flow:
# --------------------------------------
# scrape image
# imsave the image
# read the image with cv2
# do the thing on the image with function
# delete the image with os.remove()
# --------------------------------------

# site towards data science guy when putting on git 
def centroids_from_image_url(url,num_clusters):
    
    # read in our image from online and download to local computer
    r = requests.get(url, allow_redirects=True)
    open("image_from_internet.jpg", 'wb').write(r.content)

    # read in image for processing
    image = cv2.imread("image_from_internet.jpg")
    im_rgb = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        
    # SCALE IMAGE
    mat = cv2.resize(im_rgb, dsize = (60,60))

    # reshape image to pass into K Means 
    img = mat.reshape((mat.shape[0]*mat.shape[1],3))

    # apply kmeans to reshaped image
    km = KMeans(n_clusters = num_clusters)
    km_to_im = km.fit(img)

    # get the clusters
    # labels=list(km.labels_)
    centroids = km.cluster_centers_
    os.remove("image_from_internet.jpg")
    

    return centroids

In [16]:
# check if all URLS are valid with l test run then full URLS list
l = [URLS[0], " http://nosidngsoidngisodngsgsfs.com",URLS[1]]

for u in l:    
    try:
        r = requests.get(u, allow_redirects=True)
        open("image_from_internet.jpg", 'wb').write(r.content)

    except:
        print(u)

 http://nosidngsoidngisodngsgsfs.com


In [17]:
# compute centroids column

# THIS TAKES 10 HOURS TO RUN 
centroids_col = [centroids_from_image_url(im_url,5) for im_url in URLS]

<ipython-input-15-9cf78c4ffb13>:32: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km_to_im = km.fit(img)
<ipython-input-15-9cf78c4ffb13>:32: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km_to_im = km.fit(img)
<ipython-input-15-9cf78c4ffb13>:32: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km_to_im = km.fit(img)
<ipython-input-15-9cf78c4ffb13>:32: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km_to_im = km.fit(img)
<ipython-input-15-9cf78c4ffb13>:32: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  km_to_im = km.fit(img)
<ipython-input-15-9cf78c4ffb13>:32: ConvergenceWarning: Number of

In [5]:
# game_names = list(df_0['name'])
# centroids_dict = {game_names[i]: centroids_from_image_url(URLS[i],5) for i in range(len(URLS))}

In [ ]:
# split into dominant colors to put into data frame FROM DICTIONARY
# games_names_to_merge_on = list(centroids_dict.keys())
# color_list = list(centroids_dict.values())
# color_1 = [color[0] for color in color_list]
# color_2 = [color[1] for color in color_list]
# color_3 = [color[2] for color in color_list]
# color_4 = [color[3] for color in color_list]
# color_5 = [color[4] for color in color_list]

In [18]:
# split into dominant colors to put into data frame 
color_1 = [color[0] for color in centroids_col]
color_2 = [color[1] for color in centroids_col]
color_3 = [color[2] for color in centroids_col]
color_4 = [color[3] for color in centroids_col]
color_5 = [color[4] for color in centroids_col]

In [19]:
def rgb2color(color_list):
    n = len(color_list)
    
    grey_cut_off = 3
    black_cut_off = 60
    white_cut_off = 240
    
    for i in range(n):
        # generalize very common colors first, being black,white,grey
        if floor(color_list[i][0]) < black_cut_off and floor(color_list[i][1]) < black_cut_off and floor(color_list[i][2]) < black_cut_off:
            color_list[i] = "Black"
        elif floor(color_list[i][0]) > white_cut_off and floor(color_list[i][1]) > white_cut_off and floor(color_list[i][2]) > white_cut_off:
            color_list[i] = "White"
        elif floor(color_list[i][2]) == floor(color_list[i][0]) and floor(color_list[i][0]) == floor(color_list[i][2]) and floor(color_list[i][1]) == floor(color_list[i][2]):
            color_list[i] = "Grey"
        elif abs(floor(color_list[i][0])-floor(color_list[i][1])) <= grey_cut_off or abs(floor(color_list[i][0])-floor(color_list[i][2])) <= grey_cut_off or abs(floor(color_list[i][1])-floor(color_list[i][2])) <= grey_cut_off:
            color_list[i] = "Grey"
        
        # generalize primary colors 
        elif floor(color_list[i][1]) > floor(color_list[i][0]) and floor(color_list[i][1]) > floor(color_list[i][2]):
            color_list[i] = "Green/Blue Green/Aqua"
        # elif floor(color_list[i][1]) > floor(color_list[i][0]) and floor(color_list[i][1]) > floor(color_list[i][2]):
        #     color_list[i] = "Blue Green/Aqua"
        elif floor(color_list[i][0]) > floor(color_list[i][2]) and floor(color_list[i][1]) > floor(color_list[i][2]) and floor(color_list[i][0]) > floor(color_list[i][1]):
            color_list[i] = "Orange/Yellow/Brown"
        elif floor(color_list[i][0]) > floor(color_list[i][1]) and floor(color_list[i][0]) > floor(color_list[i][2]):
            color_list[i] = "Red/Pink"
        elif floor(color_list[i][0]) > floor(color_list[i][1]) and floor(color_list[i][2]) > floor(color_list[i][1]):
            color_list[i] = "Purple"
        elif floor(color_list[i][2]) > floor(color_list[i][0]) and floor(color_list[i][2]) > floor(color_list[i][1]):
            color_list[i] = "Blue"
        
    return color_list

In [20]:
color_1_string = rgb2color(color_1)
color_2_string = rgb2color(color_2)
color_3_string = rgb2color(color_3)
color_4_string = rgb2color(color_4)
color_5_string = rgb2color(color_5)

In [ ]:
# Data Frame with Game Name to Merge on
# top_5_colors_with_name_df = pd.DataFrame({'name':games_names_to_merge_on,'Color_1':color_1_string,'Color_2':color_2_string,'Color_3':color_3_string,'Color_4':color_4_string,'Color_5':color_5_string})
# top_5_colors_with_name_df.to_csv("Cover_Art_Colors.csv")

In [ ]:
# Merge Cover_Art_Colors.csv with boris_data.csv
# final_df = df_0.merge(top_5_colors_with_name_df, on = 'name')
# final_df.to_csv("DONE.csv")

In [22]:
top_5_colors_df = pd.DataFrame({'Color_1':color_1_string,'Color_2':color_2_string,'Color_3':color_3_string,'Color_4':color_4_string,'Color_5':color_5_string})
top_5_colors_df.to_csv("done.csv")

In [9]:
# df_0['Color 1'] = color_1_string
# df_0['Color 2'] = color_2_string
# df_0['Color 3'] = color_3_string
# df_0['Color 4'] = color_4_string
# df_0['Color 5'] = color_5_string

In [ ]:
# add 5 columns to df_0
# to_dum_dum = "df0 WITH COLUMNS"
# dum_dum = pd.get_dummies(to_dum_dum)
# dum_dum.to_csv("Complete_Data.csv")

In [6]:
colorDF = pd.read_csv("done.csv")
clean_df['Color 1'] = colorDF['Color_1']
clean_df['Color 2'] = colorDF['Color_2']
clean_df['Color 3'] = colorDF['Color_3']
clean_df['Color 4'] = colorDF['Color_4']
clean_df['Color 5'] = colorDF['Color_5']
clean_df.to_csv("Final_Data_w_Colors.csv")